# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 1

### Introdução

Em projetos de ciência de dados visando a construção de modelos de *machine learning*, ou aprendizado estatístico, é muito incomum que os dados iniciais estejam já no formato ideal para a construção de modelos. São necessários vários passos intermediários de pré-processamento de dados, como por exemplo a codificação de variáveis categóricas, normalização de variáveis numéricas, tratamento de dados faltantes, etc. A biblioteca **scikit-learn** -- uma das mais populares bibliotecas de código-aberto para *machine learning* no mundo -- possui diversas funções já integradas para a realização das transformações de dados mais utilizadas. Entretanto, em um fluxo comum de um modelo de aprendizado de máquina, é necessária a aplicação dessas transformações pelo menos duas vezes: a primeira vez para "treinar" o modelo, e depois novamente quando novos dados forem enviados como entrada para serem classificados por este modelo. 

Para facilitar o trabalho com esse tipo de fluxo, o scikit-learn possui também uma ferramenta chamada **Pipeline**, que nada mais é do que uma lista ordenada de transformações que devem ser aplicadas nos dados. Para auxiliar no desenvolvimento e no gerenciamento de todo o ciclo-de-vida dessas aplicações, alem do uso de Pipelines, as equipes de cientistas de dados podem utilizar em conjunto o **Watson Machine Learning**, que possui dezenas de ferramentas para treinar, gerenciar, hospedar e avaliar modelos baseados em aprendizado de máquina. Além disso, o Watson Machine Learning é capaz de encapsular pipelines e modelos em uma API pronta para uso e integração com outras aplicações.

Durante o desafio 2, você participante irá aprender a construir uma **Pipeline** para um modelo de classificação e hospedá-lo como uma API com o auxílio do Watson Machine Learning. Uma vez hospedado, você poderá integrar o modelo criado com outras aplicações, como assistentes virtuais e muito mais. Neste notebook, será apresentado um exemplo funcional de criação de um modelo e de uma pipeline no scikit-learn (que você poderá utilizar como template para a sua solução!).

### Importando bibliotecas

In [1]:
# Em seguida iremos importar diversas bibliotecas que serão utilizadas:

# Pacote para trabalhar com JSON
import json

# Pacote para realizar requisições HTTP
import requests

# Pacote para exploração e análise de dados
import pandas as pd

# Pacote com métodos numéricos e representações matriciais
import numpy as np

# Pacote para construção de modelo baseado na técnica Gradient Boosting
import xgboost as xgb

# Pacotes do scikit-learn para pré-processamento de dados
# "SimpleImputer" é uma transformação para preencher valores faltantes em conjuntos de dados
from sklearn.impute import SimpleImputer

# Pacotes do scikit-learn para treinamento de modelos e construção de pipelines
# Método para separação de conjunto de dados em amostras de treino e teste
from sklearn.model_selection import train_test_split
# Método para criação de modelos baseados em árvores de decisão
from sklearn.tree import DecisionTreeClassifier
# Classe para a criação de uma pipeline de machine-learning
from sklearn.pipeline import Pipeline

# Pacotes do scikit-learn para avaliação de modelos
# Métodos para validação cruzada do modelo criado
from sklearn.model_selection import KFold, cross_validate

In [2]:
df_data_1 = pd.read_csv('../Data/dataset_desafio_2.csv')

Temos 15 colunas presentes no dataset fornecido, sendo dezessete delas variáveis características (dados de entrada) e um delas uma variável-alvo (que queremos que o nosso modelo seja capaz de prever). 

As variáveis características são:

    MATRICULA       - número de matrícula do estudante
    NOME            - nome completo do estudante
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    REPROVACOES_GO  - número de reprovações na disciplina de ``Gestão Operacional``
    NOTA_DE         - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    NOTA_EM         - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    NOTA_MF         - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    NOTA_GO         - média simples das notas do aluno na disciplina de ``Gestão Operacional`` (0-10)
    INGLES          - variável binária que indica se o estudante tem conhecimento em língua inglesa (0 -> sim ou 1 -> não).
    H_AULA_PRES     - horas de estudo presencial realizadas pelo estudante
    TAREFAS_ONLINE  - número de tarefas online entregues pelo estudante
    FALTAS          - número de faltas acumuladas do estudante (todas disciplinas)
    
A variável-alvo é:

    PERFIL               - uma *string* que indica uma de cinco possibilidades: 
        "EXCELENTE"      - Estudante não necessita de mentoria
        "MUITO BOM"      - Estudante não necessita de mentoria
        "HUMANAS"        - Estudante necessita de mentoria exclusivamente em matérias com conteúdo de ciências humanas
        "EXATAS"         - Estudante necessita de mentoria apenas em disciplinas com conteúdo de ciências exatas
        "DIFICULDADE"    - Estudante necessita de mentoria em duas ou mais disciplinas
        
Com um modelo capaz de classificar um estudante em uma dessas categorias, podemos automatizar parte da mentoria estudantil através de assistentes virtuais, que serão capazes de recomendar práticas de estudo e conteúdo personalizado com base nas necessidades de cada aluno.

### Explorando os dados fornecidos

Podemos continuar a exploração dos dados fornecidos com a função ``info()``:

In [4]:
df_data_1.head()

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


#### Removendo colunas desnecessárias

In [7]:
columns_to_drop = ['NOME','MATRICULA']
df_data_2 = df_data_1.drop(labels=columns_to_drop, axis='columns')

#### Tratando a ausência dos dados

In [13]:
# Verificando se existem valores nulos
existsNull =  df_data_2.isnull().values.any()

# Em alguns caso, pode ser necessário
#existsNull = (df_data_2=='-nan(ind)').values.any()

# Verificando a porcentagem de valores nulos
percentageNull = df_data_2.isnull().values.sum()/len(df_data_2)

print('Existem valores nulos?' , existsNull)
print('Porcentagem de linhas com valores nulos:' , percentageNull)

Existem valores nulos? True
Porcentagem de linhas com valores nulos: 0.3672


In [21]:
# 5.Filling with a value
df_data_3 = df_data_2.fillna(value='0')

# Verificando se existem valores nulos
existsNull =  df_data_3.isnull().values.any()

# Verificando a porcentagem de valores nulos
percentageNull = df_data_3.isnull().values.sum()/len(df_data_3)

print('Existem valores nulos?' , existsNull)
print('Porcentagem de linhas com valores nulos:' , percentageNull)

Existem valores nulos? False
Porcentagem de linhas com valores nulos: 0.0


#### Seleção de atributos

In [ ]:
## Code here

#### Dividindo o banco

In [73]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, val_index in split.split(df_data_3, df_data_3.PERFIL):
    data_train = df_data_3.loc[train_index]    
    data_val = df_data_3.loc[val_index]
    
print(data_train.shape)
print(data_val.shape)

(16000, 13)
(4000, 13)


#### Separando features de target


In [98]:
from sklearn import preprocessing

# Separate label and features
columns = data_train.columns
columns = columns.drop("PERFIL")

# Train
X = data_train.iloc[:, :-1].values
X = pd.DataFrame(X)
X.columns = [columns]

Y = data_train.iloc[:, -1].values
Y = pd.DataFrame(Y)
Y.columns = ['PERFIL']

#le = preprocessing.LabelEncoder()
#Y = le.fit_transform(Y)
#Y = pd.DataFrame(Y)

#perfil_map = {'EXCELENTE' : 0, 'DIFICULDADE' : 1, 'HUMANAS': 2, 'EXATAS': 3, 'MUITO BOM': 4}
# Aplicando o mapeamento ao dataset
#Y.PERFIL = Y.PERFIL.map(perfil_map)

# Validation

X_val = data_val.iloc[:, :-1].values
y_val = data_val.iloc[:, -1].values
y_val = le.fit_transform(y_test)

### Model Selection

#### Decision Tree Classifier

In [99]:
# Criação da árvore de decisão com a biblioteca ``scikit-learn``:
dtc_model = DecisionTreeClassifier()  # O modelo será criado com os parâmetros padrões da biblioteca

# Treino do modelo (é chamado o método *fit()* com os conjuntos de treino)
dtc_model.fit(
    X,
    Y
)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [100]:
# Realização de teste cego no modelo criado
y_pred = dtc_model.predict(X_val)
y_pred = le.fit_transform(y_pred)

In [101]:
from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_val, y_pred), 2)))

Acurácia: 72.0%


#### Logistic Regression

In [97]:
from sklearn.linear_model import LogisticRegression

classifier_lr = LogisticRegression(random_state = 0, solver='liblinear')
classifier_lr.fit(X, Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [104]:
# Realização de teste cego no modelo criado
y_pred = classifier_lr.predict(X_val)
y_pred = le.fit_transform(y_pred)

In [105]:
from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_val, y_pred), 2)))

Acurácia: 75.0%


#### Random Forest

In [110]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X, Y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [111]:
# Realização de teste cego no modelo criado
y_pred = classifier_rf.predict(X_val)
y_pred = le.fit_transform(y_pred)

In [112]:
from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_val, y_pred), 2)))

Acurácia: 81.0%


#### Cat Boost

In [119]:
from catboost import CatBoostClassifier
classifier_cb = CatBoostClassifier()
classifier_cb.fit(X, Y)

Learning rate set to 0.091145
0:	learn: 1.3950000	total: 5.91ms	remaining: 5.9s
1:	learn: 1.2449719	total: 12ms	remaining: 6.01s
2:	learn: 1.1300309	total: 18.4ms	remaining: 6.13s
3:	learn: 1.0428524	total: 23.6ms	remaining: 5.86s
4:	learn: 0.9714380	total: 30.8ms	remaining: 6.13s
5:	learn: 0.9105878	total: 36.4ms	remaining: 6.03s
6:	learn: 0.8590206	total: 42.5ms	remaining: 6.03s
7:	learn: 0.8138469	total: 48.2ms	remaining: 5.97s
8:	learn: 0.7752758	total: 53.9ms	remaining: 5.93s
9:	learn: 0.7414798	total: 59.6ms	remaining: 5.9s
10:	learn: 0.7119842	total: 65.7ms	remaining: 5.91s
11:	learn: 0.6878713	total: 71.2ms	remaining: 5.86s
12:	learn: 0.6653499	total: 77ms	remaining: 5.85s
13:	learn: 0.6444151	total: 83ms	remaining: 5.84s
14:	learn: 0.6274376	total: 89.1ms	remaining: 5.85s
15:	learn: 0.6112207	total: 96.1ms	remaining: 5.91s
16:	learn: 0.5964389	total: 101ms	remaining: 5.84s
17:	learn: 0.5827880	total: 107ms	remaining: 5.81s
18:	learn: 0.5702762	total: 113ms	remaining: 5.84s
19:

177:	learn: 0.4103388	total: 1.07s	remaining: 4.92s
178:	learn: 0.4101599	total: 1.07s	remaining: 4.92s
179:	learn: 0.4099512	total: 1.08s	remaining: 4.92s
180:	learn: 0.4096223	total: 1.09s	remaining: 4.92s
181:	learn: 0.4094214	total: 1.09s	remaining: 4.92s
182:	learn: 0.4092487	total: 1.1s	remaining: 4.92s
183:	learn: 0.4089558	total: 1.11s	remaining: 4.93s
184:	learn: 0.4086797	total: 1.12s	remaining: 4.93s
185:	learn: 0.4084941	total: 1.12s	remaining: 4.92s
186:	learn: 0.4083567	total: 1.14s	remaining: 4.96s
187:	learn: 0.4081871	total: 1.15s	remaining: 4.96s
188:	learn: 0.4079519	total: 1.16s	remaining: 4.96s
189:	learn: 0.4077852	total: 1.16s	remaining: 4.96s
190:	learn: 0.4076115	total: 1.18s	remaining: 4.99s
191:	learn: 0.4075101	total: 1.19s	remaining: 4.99s
192:	learn: 0.4073281	total: 1.19s	remaining: 4.98s
193:	learn: 0.4070477	total: 1.2s	remaining: 4.98s
194:	learn: 0.4069234	total: 1.21s	remaining: 4.98s
195:	learn: 0.4066141	total: 1.21s	remaining: 4.98s
196:	learn: 0.

337:	learn: 0.3801545	total: 2.86s	remaining: 5.6s
338:	learn: 0.3799483	total: 2.87s	remaining: 5.6s
339:	learn: 0.3799130	total: 2.88s	remaining: 5.6s
340:	learn: 0.3798235	total: 2.89s	remaining: 5.59s
341:	learn: 0.3795891	total: 2.9s	remaining: 5.59s
342:	learn: 0.3792367	total: 2.92s	remaining: 5.58s
343:	learn: 0.3790386	total: 2.92s	remaining: 5.58s
344:	learn: 0.3788560	total: 2.94s	remaining: 5.58s
345:	learn: 0.3787463	total: 2.95s	remaining: 5.57s
346:	learn: 0.3784872	total: 2.96s	remaining: 5.57s
347:	learn: 0.3782763	total: 2.97s	remaining: 5.56s
348:	learn: 0.3781031	total: 2.98s	remaining: 5.56s
349:	learn: 0.3779729	total: 2.99s	remaining: 5.55s
350:	learn: 0.3778466	total: 3s	remaining: 5.55s
351:	learn: 0.3776274	total: 3.01s	remaining: 5.54s
352:	learn: 0.3774842	total: 3.02s	remaining: 5.54s
353:	learn: 0.3773525	total: 3.03s	remaining: 5.53s
354:	learn: 0.3771612	total: 3.04s	remaining: 5.53s
355:	learn: 0.3769311	total: 3.05s	remaining: 5.52s
356:	learn: 0.37679

502:	learn: 0.3532953	total: 4.83s	remaining: 4.78s
503:	learn: 0.3530951	total: 4.85s	remaining: 4.78s
504:	learn: 0.3530060	total: 4.86s	remaining: 4.77s
505:	learn: 0.3528872	total: 4.87s	remaining: 4.75s
506:	learn: 0.3526425	total: 4.88s	remaining: 4.75s
507:	learn: 0.3524860	total: 4.89s	remaining: 4.74s
508:	learn: 0.3523010	total: 4.9s	remaining: 4.73s
509:	learn: 0.3521717	total: 4.91s	remaining: 4.72s
510:	learn: 0.3520526	total: 4.92s	remaining: 4.71s
511:	learn: 0.3519114	total: 4.93s	remaining: 4.7s
512:	learn: 0.3516423	total: 4.94s	remaining: 4.69s
513:	learn: 0.3515227	total: 4.96s	remaining: 4.69s
514:	learn: 0.3514134	total: 4.97s	remaining: 4.68s
515:	learn: 0.3512726	total: 4.98s	remaining: 4.67s
516:	learn: 0.3511454	total: 4.99s	remaining: 4.66s
517:	learn: 0.3510003	total: 5s	remaining: 4.65s
518:	learn: 0.3508404	total: 5.01s	remaining: 4.65s
519:	learn: 0.3507314	total: 5.03s	remaining: 4.64s
520:	learn: 0.3505815	total: 5.04s	remaining: 4.64s
521:	learn: 0.350

662:	learn: 0.3319646	total: 6.62s	remaining: 3.37s
663:	learn: 0.3318722	total: 6.63s	remaining: 3.36s
664:	learn: 0.3316518	total: 6.66s	remaining: 3.35s
665:	learn: 0.3315697	total: 6.67s	remaining: 3.34s
666:	learn: 0.3313598	total: 6.68s	remaining: 3.33s
667:	learn: 0.3313110	total: 6.69s	remaining: 3.32s
668:	learn: 0.3311431	total: 6.7s	remaining: 3.31s
669:	learn: 0.3310496	total: 6.71s	remaining: 3.3s
670:	learn: 0.3309171	total: 6.72s	remaining: 3.29s
671:	learn: 0.3308266	total: 6.73s	remaining: 3.28s
672:	learn: 0.3306848	total: 6.74s	remaining: 3.27s
673:	learn: 0.3305244	total: 6.75s	remaining: 3.26s
674:	learn: 0.3303138	total: 6.76s	remaining: 3.25s
675:	learn: 0.3302019	total: 6.77s	remaining: 3.24s
676:	learn: 0.3299947	total: 6.78s	remaining: 3.24s
677:	learn: 0.3298654	total: 6.79s	remaining: 3.23s
678:	learn: 0.3296809	total: 6.8s	remaining: 3.21s
679:	learn: 0.3295572	total: 6.81s	remaining: 3.21s
680:	learn: 0.3294974	total: 6.82s	remaining: 3.19s
681:	learn: 0.3

828:	learn: 0.3113996	total: 8.53s	remaining: 1.76s
829:	learn: 0.3112629	total: 8.55s	remaining: 1.75s
830:	learn: 0.3111235	total: 8.56s	remaining: 1.74s
831:	learn: 0.3109973	total: 8.57s	remaining: 1.73s
832:	learn: 0.3109511	total: 8.59s	remaining: 1.72s
833:	learn: 0.3108514	total: 8.6s	remaining: 1.71s
834:	learn: 0.3107554	total: 8.61s	remaining: 1.7s
835:	learn: 0.3106361	total: 8.64s	remaining: 1.69s
836:	learn: 0.3105210	total: 8.65s	remaining: 1.68s
837:	learn: 0.3103612	total: 8.66s	remaining: 1.67s
838:	learn: 0.3101814	total: 8.67s	remaining: 1.66s
839:	learn: 0.3100855	total: 8.68s	remaining: 1.65s
840:	learn: 0.3099526	total: 8.69s	remaining: 1.64s
841:	learn: 0.3098014	total: 8.72s	remaining: 1.64s
842:	learn: 0.3097131	total: 8.73s	remaining: 1.63s
843:	learn: 0.3095936	total: 8.74s	remaining: 1.61s
844:	learn: 0.3094402	total: 8.75s	remaining: 1.6s
845:	learn: 0.3092540	total: 8.76s	remaining: 1.59s
846:	learn: 0.3091900	total: 8.78s	remaining: 1.58s
847:	learn: 0.3

990:	learn: 0.2940956	total: 10.5s	remaining: 95.5ms
991:	learn: 0.2939899	total: 10.5s	remaining: 84.9ms
992:	learn: 0.2938645	total: 10.5s	remaining: 74.3ms
993:	learn: 0.2937229	total: 10.6s	remaining: 63.7ms
994:	learn: 0.2935539	total: 10.6s	remaining: 53.1ms
995:	learn: 0.2934119	total: 10.6s	remaining: 42.5ms
996:	learn: 0.2932797	total: 10.6s	remaining: 31.9ms
997:	learn: 0.2931643	total: 10.6s	remaining: 21.2ms
998:	learn: 0.2930837	total: 10.6s	remaining: 10.6ms
999:	learn: 0.2929027	total: 10.6s	remaining: 0us


In [117]:
# Realização de teste cego no modelo criado
y_pred = classifier_cb.predict(X_val)
y_pred = le.fit_transform(y_pred)

In [118]:
from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_val, y_pred), 2)))

Acurácia: 81.0%
